<a href="https://colab.research.google.com/github/PalakDograCSE-AI2004/Plant-Disease-Detection__SummerInternship/blob/main/PROJECT_PDD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# project plant disease detection

In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from sklearn import svm, neighbors, ensemble, tree, naive_bayes
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import torchvision.models as models
from torchvision import transforms
import numpy as np
from PIL import Image
import zipfile
import os

import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

import torch.optim as optim
from torch.utils.data import Dataset
#import timm
import torchvision


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
zip_file_path ="drive/MyDrive/archive.zip"
extract_path ="drive/MyDrive/plantdiseasedetection"


with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [ ]:
# 1. Data Augmentation

transform = transforms.Compose([
    transforms.RandomRotation(degrees=20),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.Resize((224, 224)),  # Resize images to a consistent size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 2. Load and Preprocess Data

data_path = "drive/MyDrive/plantdiseasedetection"
dataset = ImageFolder(data_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
# 3. Splitting the Data

total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

train_set, val_set, test_set = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32)
test_loader = DataLoader(test_set, batch_size=32)

In [ ]:
# 4. Feature Extraction (Using CNN Features)
model = models.resnet18(pretrained=True)
model = torch.nn.Sequential(*list(model.children())[:-1])  # Remove the last FC layer

def extract_features(loader):
    model.eval()
    features = []
    labels = []
    with torch.no_grad():
        for inputs, targets in loader:
            outputs = model(inputs)  # Use tensor inputs directly
            features.extend(outputs)
            labels.extend(targets)
    return features, labels

train_features, train_labels = extract_features(train_loader)
val_features, val_labels = extract_features(val_loader)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 94.4MB/s]


In [ ]:
train_features_numpy = torch.stack(train_features).numpy()
val_features_numpy = torch.stack(val_features).numpy()

In [ ]:
train_features_flattened = train_features_numpy.reshape(len(train_features_numpy), -1)
val_features_flattened = val_features_numpy.reshape(len(val_features_numpy), -1)

In [ ]:
train_labels_numpy = np.array(train_labels)
val_labels_numpy = np.array(val_labels)

In [ ]:
# Defining the parameters grid for GridSearchCV
param_grid={'C':[0.1,1,10,100],
            'gamma':[0.0001,0.001,0.1,1],
            'kernel':['rbf','poly']}

# Creating a support vector classifier
svc=svm.SVC(probability=True)

# Creating a model using GridSearchCV with the parameters grid
model=GridSearchCV(svc,param_grid)

In [ ]:
# Training the model using the training data
model.fit(train_features_flattened, train_labels_numpy)



GridSearchCV(estimator=SVC(probability=True),
             param_grid={'C': [0.1, 1, 10, 100],
                         'gamma': [0.0001, 0.001, 0.1, 1],
                         'kernel': ['rbf', 'poly']})

In [ ]:
# Testing the model using the testing data
y_pred = model.predict(val_features_flattened)

# Calculating the accuracy of the model
accuracy = accuracy_score(val_labels_numpy, y_pred)

# Print the accuracy of the model
print(f"The model is {accuracy*100}% accurate")

The model is 99.73821989528795% accurate


In [ ]:
# Create a KNN classifier
knn = KNeighborsClassifier()  # You can adjust the number of neighbors (k)

# Train the KNN model on the training data
knn.fit(train_features_flattened, train_labels_numpy)

# Testing the KNN model using the validation data
y_pred_knn = knn.predict(val_features_flattened)

# Calculating the accuracy of the KNN model
accuracy_knn = accuracy_score(val_labels_numpy, y_pred_knn)

# Print the accuracy of the KNN model
print(f"The KNN model is {accuracy_knn*100}% accurate")


The KNN model is 97.90575916230367% accurate


In [ ]:
rf_model = ensemble.RandomForestClassifier()
rf_model.fit(train_features_flattened, train_labels_numpy)
rf_predictions = rf_model.predict(val_features_flattened)
rf_accuracy = accuracy_score(val_labels_numpy, rf_predictions)
print(f"The Random Forest is {rf_accuracy*100}% accurate")

The Random Forest is 98.95287958115183% accurate


In [ ]:
dt_model = tree.DecisionTreeClassifier()
dt_model.fit(train_features_flattened, train_labels_numpy)
dt_predictions = dt_model.predict(val_features_flattened)
dt_accuracy = accuracy_score(val_labels_numpy, dt_predictions)
print(f"The Decision Tree is {dt_accuracy*100}% accurate")

The Decision Tree is 98.29842931937172% accurate


In [ ]:
nb_model = naive_bayes.GaussianNB()
nb_model.fit(train_features_flattened, train_labels_numpy)
nb_predictions = nb_model.predict(val_features_flattened)
nb_accuracy = accuracy_score(val_labels_numpy, nb_predictions)
print(f"The Naive Bayes is {nb_accuracy*100}% accurate")

The Naive Bayes is 95.02617801047121% accurate


'''class ImageClassificationBase(nn.Module):

    def training_step(self, batch):
        images, labels = batch
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss

    def validation_step(self, batch):
        images, labels = batch
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}

    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))'''


In [ ]:
#CNN Model

In [ ]:
# Create a custom dataset class
class PlantDiseaseDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.images = []
        self.labels = []

        # Load data from subdirectories
        for label in os.listdir(data_dir):
            label_dir = os.path.join(data_dir, label)
            for image_file in os.listdir(label_dir):
                self.images.append(os.path.join(label_dir, image_file))
                self.labels.append(label)

    def _len_(self):
        return len(self.images)

    def _getitem_(self, idx):
        image = Image.open(self.images[idx])
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Create dataset and dataloaders
dataset = PlantDiseaseDataset(data_dir=data_path, transform=transform)

In [ ]:
model = models.resnet18(pretrained=True)  # You can use other pre-trained models as well
num_classes = 30  # Number of disease classes

# Modify the final fully connected layer to match the number of classes
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, num_classes)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

# Save the trained model
torch.save(model.state_dict(), "plant_disease_model.pth")

Epoch 1/10, Loss: 0.09052714748395374
Epoch 2/10, Loss: 0.02388353709875446
Epoch 3/10, Loss: 0.00569027583622171
Epoch 4/10, Loss: 0.008434237002721343
Epoch 5/10, Loss: 0.009352539188122526
Epoch 6/10, Loss: 0.001573430643393944
Epoch 7/10, Loss: 0.002946820953671444
Epoch 8/10, Loss: 0.032009144067144134
Epoch 9/10, Loss: 0.01013190427823117
Epoch 10/10, Loss: 0.004241926611476694


In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation Accuracy: {100 * correct / total}%")

Validation Accuracy: 100.0%


In [ ]:
!pip install scikit-learn
from sklearn.metrics import precision_score, recall_score, f1_score


In [ ]:
# Evaluate SVM model
svm_accuracy = accuracy_score(val_labels_numpy, y_pred)
svm_precision = precision_score(val_labels_numpy, y_pred)
svm_recall = recall_score(val_labels_numpy, y_pred)
svm_f1 = f1_score(val_labels_numpy, y_pred)

# Evaluate KNN model
knn_accuracy = accuracy_score(val_labels_numpy, y_pred_knn)
knn_precision = precision_score(val_labels_numpy, y_pred_knn)
knn_recall = recall_score(val_labels_numpy, y_pred_knn)
knn_f1 = f1_score(val_labels_numpy, y_pred_knn)

# Evaluate Random Forest model
rf_accuracy = accuracy_score(val_labels_numpy, rf_predictions)
rf_precision = precision_score(val_labels_numpy, rf_predictions)
rf_recall = recall_score(val_labels_numpy, rf_predictions)
rf_f1 = f1_score(val_labels_numpy, rf_predictions)

# Evaluate Decision Tree model
dt_accuracy = accuracy_score(val_labels_numpy, dt_predictions)
dt_precision = precision_score(val_labels_numpy, dt_predictions)
dt_recall = recall_score(val_labels_numpy, dt_predictions)
dt_f1 = f1_score(val_labels_numpy, dt_predictions)

# Evaluate Naive Bayes model
nb_accuracy = accuracy_score(val_labels_numpy, nb_predictions)
nb_precision = precision_score(val_labels_numpy, nb_predictions)
nb_recall = recall_score(val_labels_numpy, nb_predictions)
nb_f1 = f1_score(val_labels_numpy, nb_predictions)


# Print performance metrics
print("Performance Metrics:")
print(f"SVM: Accuracy={svm_accuracy}, Precision={svm_precision}, Recall={svm_recall}, F1-score={svm_f1}")
print(f"KNN: Accuracy={knn_accuracy}, Precision={knn_precision}, Recall={knn_recall}, F1-score={knn_f1}")
print(f"Random Forest: Accuracy={rf_accuracy}, Precision={rf_precision}, Recall={rf_recall}, F1-score={rf_f1}")
print(f"Decision Tree: Accuracy={dt_accuracy}, Precision={dt_precision}, Recall={dt_recall}, F1-score={dt_f1}")
print(f"Naive Bayes: Accuracy={nb_accuracy}, Precision={nb_precision}, Recall={nb_recall}, F1-score={nb_f1}")


Performance Metrics:
SVM: Accuracy=0.9973821989528796, Precision=0.9974554707379135, Recall=0.9974554707379135, F1-score=0.9974554707379135
KNN: Accuracy=0.9790575916230366, Precision=0.960880195599022, Recall=1.0, F1-score=0.9800498753117207
Random Forest: Accuracy=0.9895287958115183, Precision=0.9824561403508771, Recall=0.9974554707379135, F1-score=0.98989898989899
Decision Tree: Accuracy=0.9829842931937173, Precision=0.9846938775510204, Recall=0.9821882951653944, F1-score=0.9834394904458599
Naive Bayes: Accuracy=0.9502617801047121, Precision=0.9382716049382716, Recall=0.9669211195928753, F1-score=0.9523809523809523


In [ ]:
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming you have your data ready, val_loader, val_labels

# Function to evaluate the model
def evaluate_model(model, dataloader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return np.array(all_preds), np.array(all_labels)

# Assuming your model is already defined and trained

# Evaluate the model
y_pred, y_true = evaluate_model(model, val_loader)

# Calculate performance metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

# Print performance metrics
print("Performance Metrics:")
print(f"Accuracy: {accuracy:.16f}")
print(f"Precision: {precision:.16f}")
print(f"Recall: {recall:.16f}")
print(f"F1-score: {f1:.16f}")


Performance Metrics:
Accuracy: 1.0000000000000000
Precision: 1.0000000000000000
Recall: 1.0000000000000000
F1-score: 1.0000000000000000


In [ ]:
# Print performance metrics
print("Performance Metrics:")
print(f"SVM: Accuracy={svm_accuracy}, Precision={svm_precision}, Recall={svm_recall}, F1-score={svm_f1}")
print(f"KNN: Accuracy={knn_accuracy}, Precision={knn_precision}, Recall={knn_recall}, F1-score={knn_f1}")
print(f"Random Forest: Accuracy={rf_accuracy}, Precision={rf_precision}, Recall={rf_recall}, F1-score={rf_f1}")
print(f"Decision Tree: Accuracy={dt_accuracy}, Precision={dt_precision}, Recall={dt_recall}, F1-score={dt_f1}")
print(f"Naive Bayes: Accuracy={nb_accuracy}, Precision={nb_precision}, Recall={nb_recall}, F1-score={nb_f1}")
print(f"CNN: Accuracy= {accuracy:.16f}, Precision= {precision:.16f} , Recall={recall:.16f} , F1-score={f1:.16f}")


Performance Metrics:
SVM: Accuracy=0.9973821989528796, Precision=0.9974554707379135, Recall=0.9974554707379135, F1-score=0.9974554707379135
KNN: Accuracy=0.9790575916230366, Precision=0.960880195599022, Recall=1.0, F1-score=0.9800498753117207
Random Forest: Accuracy=0.9895287958115183, Precision=0.9824561403508771, Recall=0.9974554707379135, F1-score=0.98989898989899
Decision Tree: Accuracy=0.9829842931937173, Precision=0.9846938775510204, Recall=0.9821882951653944, F1-score=0.9834394904458599
Naive Bayes: Accuracy=0.9502617801047121, Precision=0.9382716049382716, Recall=0.9669211195928753, F1-score=0.9523809523809523
CNN: Accuracy= 1.0000000000000000, Precision= 1.0000000000000000 , Recall=1.0000000000000000 , F1-score=1.0000000000000000
